In [ ]:
import sys


no_of_cpu =8
max_cores =8
executor_mem = '56g'


Job_Name = 'Nishant:SPA Part-1'


from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'

spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################

conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')

conf.set('spark.ui.port','4052')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','400')
#conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('es.nodes',es_nodes)
conf.set('es.port',es_port)
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
conf.set('spark.num.executors','4')
conf.set('spark.debug.maxToStringFields', 200)

conf.set('spark.es.net.http.auth.user', es_user)
conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType



In [ ]:
from pyspark.sql.functions import hash,col,regexp_replace,collect_list,concat_ws,initcap,count,least
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

In [ ]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpcustcoe"
pwd_prod="gpcustcoe@123"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

In [ ]:
def get_latest_progress(table_name,column_name):
    
    try: 
        gscPythonOptions = {
                 "url":prod_url ,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": table_name,#table change
                 "server.port":"1150-1170"} 
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select(column_name)
        
        Max_last_run =last_run.select(max(column_name)).first()[0]


        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run))

    except Exception as E: 
        Max_last_run = datetime.datetime(2018, 10, 31, 0, 0)  # this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

In [ ]:
# loading table as dataframe
def load_gpdb_jdbc(col_str,dbtable,dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

In [ ]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123",
    "dbschema":"customermart",
    "dbtable": "header",
         "server.port":"1150-1170"}


header_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('golden_id','source_system_customer_id','active_flag','mapping_date').filter((col('active_flag')=='1')).filter((col('source_system_customer_id').isNotNull()))

In [ ]:
from pyspark.sql.window import Window

header_df = header_df.withColumn("rn", F.row_number()
        .over(Window.partitionBy('source_system_customer_id')
        .orderBy(F.col("mapping_date").desc(),F.col("golden_id").desc())))

header_df = header_df.filter(F.col("rn") == 1).drop("rn")

header_df=header_df.select('source_system_customer_id','golden_id','mapping_date')

In [ ]:
col_str=['golden_id','full_name','first_name','last_name','birth_date','gender','marital_status','weight_kg','tobacco_use_indicator','alcohol_use_indicator','height_cm']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark", "server.port":"1150-1170",
"password": "spark@456","dbschema":"customermart",
"dbtable": "customer_demographics"}
demog_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)

demog_df=demog_df.withColumn('customer_name',coalesce(col('full_name'),concat(col('first_name'),lit(' '),col('last_name'))))


In [ ]:
col_str=['golden_id','service_level']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark", "server.port":"1150-1170",
"password": "spark@456","dbschema":"customermart",
"dbtable": "customer_profile"}
profile_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)

In [ ]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123"
     ,"dbschema":"customermart"
    ,"dbtable": "address",
         "server.port":"1150-1170"}


address_retail_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('source_system_customer_id','address_type','city','last_updated')\
.filter((col('address_type')=='home'))

In [ ]:
from pyspark.sql.window import Window

address_retail_df = address_retail_df.withColumn("rn", F.row_number()
        .over(Window.partitionBy('source_system_customer_id','address_type')
        .orderBy(F.col("last_updated").desc())))

address_retail_df = address_retail_df.filter(F.col("rn") == 1).select('source_system_customer_id','city')

In [ ]:
report_date_str='2023-09-01'

In [ ]:
col_str=["policy_number",'certificate_no', 'product_name', 'product_cd', 'reference_num',
         'pol_incept_date as policy_start_date', 'pol_exp_date as policy_end_date','source_system_name',
        'source_system_customer_id', 'lob', 'sum_insured', "uw_discount", 'premiumamount as premium',
        'veh_make', 'veh_model', 'veh_no as vehicle_registration_no', 'ncb_percent','producer_cd',
        'plan_type as plan_name','status','record_type_desc','commit_timestamp as binddate','veh_engine_no','veh_chassis_no',
         'times_renewed_cnt','min_pol_incept_date','no_of_lives']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123","dbschema":"customermart","server.port":"1150-1170",
"dbtable": "policy_retail"}
policy = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
policy.columns

In [ ]:
policy=policy.filter(F.date_add(col('policy_end_date'),90)>=to_timestamp(lit(report_date_str), format='yyyy-MM-dd'))

In [ ]:
lst_cancelled=['ACDC',
'ACDE',
'APC',
'APCP',
'APD',
'APRI',
'APU',
'DCC',
'ECDU',
'ECN',
'ECR',
'EPCE',
'ERFC',
'NCN',
'RCN']

In [ ]:
policy=policy.withColumn('record_type_desc', when((col('status').isin(lst_cancelled)),lit('CANCELLATION'))\
                           .otherwise(col('record_type_desc')))

In [ ]:
#Krishnan Badrinath
policy=policy.withColumn('assets',when(col('lob').isin('2W'),'2 Wheeler')\
                                        .when(col('lob').isin('Pvt Car'),'Car')\
                                        .when(col('lob').isin('Home'),'Home').otherwise(lit('-')))

In [ ]:
w = Window.partitionBy(['policy_number','certificate_no','policy_start_date', 'policy_end_date']).orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
policy_df = policy.withColumn('row',F.row_number().over(w))
policy_df = policy_df.filter(F.col('row')==1).drop(policy_df.row)


In [ ]:
policy_df=policy_df.withColumn('certificate_no',when(col('certificate_no').isin('0','0.0','/00000','000000','00000000'),lit('00000000')).otherwise(lit(col('certificate_no'))))\
.withColumn('policy_number',lpad(col('policy_number'),10,'0'))

In [ ]:
col_str=["risk1 as relation","reference_num",'insured_name as name','insured_dob as dob']
pdrdh_df=load_gpdb_jdbc(col_str,"policy_dh_risk_details_hlt",'public')

In [ ]:
col_str=['source','scoredate', 'score','income','dc_unified_id as golden_id']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe", "server.port":"1150-1170",
"password": "gpcustcoe@123","dbschema":"datamarts",
"dbtable": "credit_bureau_data"}
credit_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
credit_df.columns

In [ ]:
col_str=["policy_no", "renewal_prop_final_buckets","segment","month"]
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
"server.port": "1151-1170",
"dbschema":"customermart",
"dbtable": "propensity_score_jan_23_jan_24"}
ps = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
ps.columns

In [ ]:
ps=ps.withColumn('month',to_date(col('month'),'MMM-yy'))

from pyspark.sql.window import Window

ps = ps.withColumn("rn", F.row_number()
        .over(Window.partitionBy('policy_no')
        .orderBy(F.col("month").desc())))

ps = ps.filter(F.col("rn") == 1).drop("rn")

ps=ps.withColumn('renewal_propensity',concat(col('segment'),lit("("),col("renewal_prop_final_buckets"),lit(")")))

ps=ps.select('policy_no','renewal_propensity','month')

propensity_df=ps.withColumn('policy_no',lpad(trim(col('policy_no')),10,'0'))

In [ ]:
propensity_df=propensity_df.selectExpr('policy_no as policy_number', 'renewal_propensity')

In [ ]:
col_str=["dc_unified_id as golden_id","'Eligible for cross sell to Health' as is_eligible"]
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
 "server.port":"1150-1170",
"dbschema":"customermart",
"dbtable": "cross_sell_eligible_cust"}
cross_sell_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
cross_sell_df.columns

In [ ]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123",
    "dbschema":"customermart"
    ,"dbtable": "contactability",
         "server.port":"1150-1170"}

contactibilty_retail_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.selectExpr('source_system_customer_id','communication_sub_type','value','last_updated','source_system_name','verification_flag').filter((col('source_system_customer_id').isNotNull()))\
.filter(col('value').isNotNull()).filter(col('source_system_name')!='CREDIT_BUREAU')

In [ ]:
contactibilty_retail_df=contactibilty_retail_df.withColumn("rnk",when(col("source_system_name")=='GC',1)\
.otherwise(2))

In [ ]:
contactibilty_retail_df=contactibilty_retail_df.withColumn('mobile',when(col('communication_sub_type')=='mobile',col('value')).otherwise(None))
contactibilty_retail_df=contactibilty_retail_df.withColumn('email',when(col('communication_sub_type')=='email',col('value')).otherwise(None))

contactibilty_retail_df = contactibilty_retail_df.withColumn('email',when((lower(trim(col("email"))).rlike("^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(\.[a-zA-Z]{2,})?$") == False)\
                                                        & (lower(trim(col("communication_sub_type")))=='email'),lit(None)).otherwise(col('email')))



contactibilty_retail_df=contactibilty_retail_df.select('source_system_customer_id','mobile','email','rnk','last_updated','verification_flag')

In [ ]:
contactibilty_retail_df.columns

In [ ]:
contactibilty_retail_df=contactibilty_retail_df.join(header_df,'source_system_customer_id','inner')

In [ ]:
contactibilty_retail_df_email=contactibilty_retail_df.filter((col('email').isNotNull())).dropDuplicates(subset=["email","golden_id"])
contactibilty_retail_df_mob=contactibilty_retail_df.filter((col('mobile').isNotNull())).drop_duplicates(subset=["mobile","golden_id"])

In [ ]:
contactibilty_retail_df_email=contactibilty_retail_df_email.withColumn('email',lower(col('email')))

In [ ]:
# from pyspark.sql.window import Window

# contactibilty_retail_df_email = contactibilty_retail_df_email.withColumn("rn", F.row_number()
#         .over(Window.partitionBy('golden_id','email')
#         .orderBy(F.col("rnk").asc(),F.col("last_updated").desc(),F.col("email").desc_nulls_last())))

# contactibilty_retail_df_email = contactibilty_retail_df_email.filter(F.col("rn")<=2)

from pyspark.sql.window import Window

contactibilty_retail_df_email = contactibilty_retail_df_email.withColumn("rn", F.dense_rank()
        .over(Window.partitionBy('golden_id')
        .orderBy(F.col("rnk").asc(),F.col("last_updated").desc(),F.col("email").desc_nulls_last())))

contactibilty_retail_df_email = contactibilty_retail_df_email.filter(F.col("rn")<=2)

In [ ]:
# from pyspark.sql.window import Window

# contactibilty_retail_df_mob = contactibilty_retail_df_mob.withColumn("rn_mob", F.row_number()
#         .over(Window.partitionBy('golden_id','mobile')
#         .orderBy(F.col("rnk").asc(),F.col("last_updated").desc(),F.col("mobile").desc_nulls_last())))

# contactibilty_retail_df_mob = contactibilty_retail_df_mob.filter(F.col("rn_mob")<=2)

from pyspark.sql.window import Window

contactibilty_retail_df_mob = contactibilty_retail_df_mob.withColumn("rn_mob", F.dense_rank()
        .over(Window.partitionBy('golden_id')
        .orderBy(F.col("rnk").asc(),F.col("last_updated").desc(),F.col("mobile").desc_nulls_last())))

contactibilty_retail_df_mob = contactibilty_retail_df_mob.filter(F.col("rn_mob")<=2)

In [ ]:
contactibilty_retail_df_email=contactibilty_retail_df_email.select('golden_id','email','rn','verification_flag').filter((col('email').isNotNull())).drop_duplicates()
contactibilty_retail_df_mob=contactibilty_retail_df_mob.selectExpr('golden_id','mobile','rn_mob','verification_flag as verification_flag_mob').filter((col('mobile').isNotNull())).drop_duplicates()

In [ ]:
contactibilty_retail_df=contactibilty_retail_df_mob.join(contactibilty_retail_df_email,'golden_id','full_outer').drop_duplicates()

In [ ]:
contactibilty_retail_df.columns

In [ ]:
contactibilty_retail_df=contactibilty_retail_df.repartition('golden_id')

In [ ]:
col_str=['claim_no','claim_feature_concat','insured_name','claim_status','paid_amt_ind as claim_net',
         'paid_date_ind as claim_closed_date'
         ,'clm_create_date as claim_date','loss_reported_date as claim_reported_date',
'policy_no as policy_number','certificate_no',"pol_incept_date as policy_start_date",
         "pol_exp_date as policy_end_date",
         'claimant_name','loss_desc_txt as loss_description','customer_id as source_system_customer_id',
'coverage_desc as coverage_type','proposal_no as num_reference_no']

gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
"dbschema":"datamarts",
"dbtable": "producer_policy_product_wise_base_data",
"server.port": "1150-1170"}
crgp_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load().selectExpr(col_str)

#df_crp=df_crp.groupby('num_reference_no').agg(countDistinct(col('claim_no')).alias('claim_cnt'),sum(col('claim_gross').cast('double')).alias('claim_gross'))

crgp_df.columns

In [ ]:
crgp_df=crgp_df.withColumn('certificate_no',when(col('certificate_no').isin('0','0.0','/00000','000000','00000000'),lit('00000000')).otherwise(lit(col('certificate_no'))))\
.withColumn('policy_number',lpad(col('policy_number'),10,'0'))

crgp_df=crgp_df.withColumn('policy_start_date',to_date(col('policy_start_date'),'dd-MM-yyyy'))\
.withColumn('policy_end_date',to_date(col('policy_end_date'),'dd-MM-yyyy'))

In [ ]:
crgp_df=crgp_df.withColumn('policy_start_date',to_date(col('policy_start_date'),'dd-MM-yyyy'))\
.withColumn('policy_end_date',to_date(col('policy_end_date'),'dd-MM-yyyy'))\
.withColumn('claim_date',to_date(col('claim_date'),'dd-MM-yyyy'))


In [ ]:
col_str=["nameofpreexistingdisease as ped","num_reference_number as reference_num"]
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
"server.port": "1151-1170",
"dbschema":"public",
"dbtable": "underwriting_dh_cnfgtr_otherdt_grid_tab_hlt"}
pd_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
pd_df.columns

In [ ]:
col_str=['feedback_title','feedback_date as date','score','golden_id']
# "current_year"
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
"server.port": "1151-1170",
"dbschema":"customermart",
"dbtable": "feedback_spa_v2"}
feedback_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
feedback_df.columns

## Active

In [ ]:
pdf=policy_df.select('source_system_customer_id','policy_end_date','record_type_desc','policy_number','certificate_no')

is_policy=header_df.join(pdf,'source_system_customer_id','inner')

In [ ]:
header_df=is_policy.select('golden_id','source_system_customer_id','mapping_date')

In [ ]:
is_policy.columns

In [ ]:
is_active_df=is_policy.withColumn('cust_status', when((col('policy_end_date') >=current_date()) &(col('record_type_desc')!='CANCELLATION') , lit(1)).otherwise(lit(0)))\

In [ ]:
is_cust_active=is_active_df.groupBy('golden_id').agg(max('cust_status').alias("is_active"))


In [ ]:
is_cust_active=is_cust_active.withColumn('is_active',when(is_cust_active['is_active']==1,True).otherwise(False))

# customerDetails

In [ ]:
demog=demog_df.select('golden_id','customer_name','birth_date','gender','marital_status')

In [ ]:
cp=demog.join(profile_df, 'golden_id',"left_outer")

In [ ]:
cust_detail=header_df.join(cp,'golden_id','inner')

In [ ]:
cust_detail=cust_detail.join(address_retail_df,'source_system_customer_id','left')

In [ ]:
from pyspark.sql import functions as F
cust_detail = cust_detail.withColumn('age', (F.months_between(current_date(), F.col('birth_date')) / 12).cast('int'))
cust_detail = cust_detail.withColumn('is_senior_citizen', when(col('age')>=60, lit(True)).otherwise(lit(False)))

In [ ]:
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"server.port":"1150-1170",
"dbschema": "public",
"dbtable": "customer_gc_genmst_customer"}
customer_gc_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('customer_code','birth_dt','dat_insert_date')

In [ ]:
cust_detail_1=cust_detail.join(customer_gc_df,customer_gc_df.customer_code==cust_detail.source_system_customer_id,'inner')\
.filter(col('age')<18)

In [ ]:
from pyspark.sql import functions as F
cust_detail_1 = cust_detail_1.withColumn('age', (F.months_between(current_date(), F.col('birth_dt')) / 12).cast('int'))

In [ ]:
cust_detail_1.columns

In [ ]:
cust_detail_age_recti=cust_detail_1.groupBy('golden_id').agg((max('age')).alias("age"))

In [ ]:
cust_detail_1=cust_detail_1.join(cust_detail_age_recti,'golden_id','inner').drop(cust_detail_1.age).drop(cust_detail_1.birth_dt).drop(cust_detail_1.dat_insert_date)

In [ ]:
cust_detail_2=cust_detail.filter((col('age')>=18)|(col('age').isNull()))

In [ ]:
cust_detail=cust_detail_2.union(cust_detail_1.select('source_system_customer_id', 'golden_id', 'mapping_date', 'customer_name', 'birth_date', 'gender', 'marital_status', 'service_level', 'city', 'age', 'is_senior_citizen'))

In [ ]:
cust_detail = cust_detail.withColumn('is_hni_customer', when(col('service_level').isin('5'), lit(True)).otherwise(lit(False)))

In [ ]:
cust_detail=cust_detail.join(is_cust_active,'golden_id','left')

In [ ]:
cust_detail.columns

In [ ]:
from pyspark.sql import functions as F
cust_detail = cust_detail.withColumn('age', (F.months_between(current_date(), F.col('birth_date')) / 12).cast('int'))
cust_detail = cust_detail.withColumn('is_senior_citizen', when(col('age')>=60, lit(True)).otherwise(lit(False)))

In [ ]:
from pyspark.sql.window import Window

w = Window.partitionBy(['golden_id']).orderBy([F.col('mapping_date').desc(),F.col('source_system_customer_id').desc()])
cust_detail = cust_detail.withColumn('row',F.row_number().over(w))
cust_detail = cust_detail.filter(F.col('row')==1).drop(cust_detail.row)

In [ ]:
cust_detail=cust_detail.join(pdf.select('policy_end_date','source_system_customer_id'),'source_system_customer_id','inner')

In [ ]:
cust_detail=cust_detail.withColumn('mapping_date',date_format(to_utc_timestamp(col('mapping_date'),"UTC"),'yyyy-MM-dd'))

In [ ]:
cust_detail.persist()
cust_detail.count()

In [ ]:
output_index = 'spa_cust_detail_v2'#cust_detail_spa_v1
schema = 'customermart'

import sys
try:
    
     cust_detail.write.option("truncate", "true").format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()
    
    
except Exception as e:
    x = e
    print(x)
else:
    x = 'success'
    print(x)

In [ ]:
# gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
#                     "user": user_prod,
#                     "password": pwd_prod,
#                     "dbschema": "customermart",
#                     "dbtable": "spa_cust_detail_v2",#policy_retail_staging_2018_03,
#                     "server.port": '1150-1170'}
# cust_detail = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()


# # .filter(~(col('product_name').isin(l)))

In [ ]:
cust_detail=cust_detail.selectExpr('golden_id','customer_name as full_name', 'gender', 'marital_status', 'service_level', 'city', 'age', 'is_senior_citizen as isseniorcitizen', 'is_hni_customer as ishnicustomer', 'is_active as isactivecustomer','mapping_date')

In [ ]:
target_index='get_spa_cust_prop_score_details_1'
print(target_index)

In [ ]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = 'get_spa_cust_prop_score_details_1'
#########delete index if already exists########################################################################################################################################################################

In [ ]:
target_index_doc_type = target_index +'/'+ target_doc_type

In [ ]:
target_index_doc_type

In [ ]:
try:
     cust_detail.write.format('org.elasticsearch.spark.sql').option('es.index.auto.create', 'true')\
        .option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type)\
        .option("es.write.operation","upsert").option("es.mapping.id","golden_id").mode("append").save() 
except Exception as e :
    x = e
else: 
    x = 200 # success
print(x)

# Feedback

In [ ]:
feedback_df=feedback_df.withColumn('feedback_date',when(to_date('date',"yyyy-MM-dd").isNotNull(), date_format(to_date('date',"yyyy-MM-dd"),"yyyy-MM-dd"))\
                                   .when(to_date('date',"MM/dd/yyyy").isNotNull(), date_format(to_date('date',"MM/dd/yyyy"),"yyyy-MM-dd"))\
                                   .when(to_date('date',"yyyy/MM/dd").isNotNull(), date_format(to_date('date',"yyyy/MM/dd"),"yyyy-MM-dd"))\
                                   .otherwise('date'))

In [ ]:
feedback_df=feedback_df.withColumn('current_year', when(year(col('feedback_date'))==year(current_date()), lit(True)).otherwise(lit(False)))


In [ ]:
from pyspark.sql.window import Window

w = Window.partitionBy(['golden_id']).orderBy([F.col('feedback_date').desc()])
feedback_df = feedback_df.withColumn('row',F.row_number().over(w))
feedback_df = feedback_df.filter(F.col('row')<=5).drop(feedback_df.row).drop('date')

In [ ]:
feedback_df=feedback_df.withColumn('score',regexp_extract('score',r"(\d+)",1).cast("int"))

In [ ]:
feedback_df=feedback_df.drop_duplicates()

In [ ]:
feedback_df.columns

In [ ]:
feedback_df=feedback_df.selectExpr('feedback_title', 'score', 'golden_id', 'feedback_date as date', 'current_year')

In [ ]:
feedback_df=feedback_df.join(cust_detail.select('golden_id'),'golden_id','inner')

In [ ]:
feedback_df.persist()
feedback_df.count()

In [ ]:
target_index='get_spa_cust_feedback'
print(target_index)

In [ ]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = 'get_spa_cust_feedback'
#########delete index if already exists########################################################################################################################################################################

In [ ]:
target_doc_type

In [ ]:
target_index_doc_type = target_index +'/'+ target_doc_type

In [ ]:
target_index_doc_type

In [ ]:
try:
         feedback_df.write.format('org.elasticsearch.spark.sql').option('es.index.auto.create', 'true')\
        .option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type)\
        .option("es.write.operation","upsert").option("es.mapping.id","golden_id").mode("append").save() 
except Exception as e :
    x = e
else: 
    x = 200 # success
print(x)

In [ ]:
feedback_df.unpersist()

# propensityDetails

In [ ]:
gid_pol=is_active_df.select('policy_number','golden_id')

In [ ]:
propensity_df.columns

In [ ]:
prop_df=gid_pol.join(propensity_df,'policy_number',"left_outer")

In [ ]:
prop_df=prop_df.join(cross_sell_df,'golden_id',"left_outer")

In [ ]:
prop_df=prop_df.selectExpr("golden_id","renewal_propensity as renewaltendency", "'-' as upselleligibility", "is_eligible as crossselleligibility")

In [ ]:
prop_df=prop_df.withColumn('renewaltendency',when(col('renewaltendency').isNull(),lit('-')).otherwise(col('renewaltendency')))\
.withColumn('crossselleligibility',when(col('crossselleligibility').isNull(),lit('-')).otherwise(col('crossselleligibility')))

In [ ]:
prop_df=prop_df.join(cust_detail.select('golden_id'),'golden_id','inner')

In [ ]:
prop_df.persist()
prop_df.count()

In [ ]:
output_index = 'prop_df_v2'#cpa_all_prod_fy20,cpa_all_prod_fy21,prop_df_f
schema = 'customermart'

import sys
try:
    
     prop_df.write.option("truncate", "true").format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()
    
    
except Exception as e:
    x = e
    print(x)
else:
    x = 'success'
    print(x)

In [ ]:
prop_df1=prop_df.groupBy('golden_id').agg(*[max(col).alias(f"{col}") for col in prop_df.columns if col!="golden_id"])

In [ ]:
target_index='get_spa_prop_score_details'
print(target_index)

In [ ]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = 'get_spa_prop_score_details'
#########delete index if already exists########################################################################################################################################################################

In [ ]:
target_index_doc_type = target_index +'/'+ target_doc_type
target_index_doc_type

In [ ]:
#########delete index ###############################################
headers = {'Content-Type': 'application/json'}
query = {"query":{"match_all":{}}}
URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index
r = requests.delete(URL, data=json.dumps(query), headers=headers) 
print(r)

In [ ]:
#########create index ##################################################################################################################################################################
headers = {'Content-Type': 'application/json'}
query_create= {"settings": { "index.number_of_shards": "1" }}
URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index
r = requests.put(URL, data=json.dumps(query_create), headers=headers)
print(r)

In [ ]:
try:
     prop_df1.write.format('org.elasticsearch.spark.sql').option('es.index.auto.create', 'true')\
        .option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type)\
        .option("es.write.operation","upsert").option("es.mapping.id","golden_id").mode("append").save() 
except Exception as e :
    x = e
else: 
    x = 200 # success
print(x)

In [ ]:
prop_df.unpersist()

# communicationDetails

In [ ]:
# com_details_df=header_df.join(contactibilty_retail_df,'source_system_customer_id',"left_outer")

In [ ]:
com_details_df=contactibilty_retail_df

In [ ]:
contactibilty_retail_df.columns

In [ ]:
com_details_df=com_details_df.withColumn('primary_mobile_no', when(col('rn_mob')==1,(col('mobile'))))\
.withColumn('primary_number_status', when((col('verification_flag_mob').isin('Y'))&(col('rn_mob')==1) &(col('primary_mobile_no').isNotNull()), lit('Verified'))\
            .when(col('primary_mobile_no').isNull(),lit('-')).otherwise(lit('Not Verified')))\
.withColumn('primary_number_status_enum', when(col('primary_number_status').isin('Verified'), lit(1))\
            .when(col('primary_number_status').isin('-'),lit('-')).otherwise(lit(0)))\
.withColumn('primary_email', when(col('rn')==1, lit(col('email'))).otherwise(lit(None)))\
.withColumn('primary_email_status', when((col('verification_flag_mob').isin('Y'))&(col('rn')==1)&(col('primary_email').isNotNull()), lit('Verified'))\
            .when(col('primary_email').isNull(),lit('-')).otherwise(lit('Not Verified')))\
.withColumn('primary_email_status_enum',when(col('primary_email_status').isin('Verified'), lit(1))\
            .when(col('primary_email_status').isin('-'),lit('-')).otherwise(lit(0)))\
.withColumn('preffered_communication_node',when(col('primary_mobile_no').isNotNull(), lit('Mobile'))\
                                        .when(col('primary_email').isNotNull(), lit('Email')).otherwise(lit('-')))\
.withColumn('prefferedLanguage',lit('-'))\
.withColumn('secondaryMobileNumber',when(col('rn_mob')==2,(col('mobile'))))\
.withColumn('secondaryEmail',when(col('rn')==2,(col('email'))))\
.withColumn('mobile_contactability',lit('-'))\
.withColumn('email_contactability',lit('-'))


In [ ]:
com_details_df=com_details_df.groupBy('golden_id').agg(*[max(col).alias(f"{col}") for col in com_details_df.columns if col!="golden_id"])

In [ ]:
com_details_df=com_details_df.select('golden_id','primary_mobile_no', 'primary_number_status', 'primary_number_status_enum', 'primary_email', 'primary_email_status', 'primary_email_status_enum', 'preffered_communication_node', 'prefferedLanguage', 'secondaryMobileNumber', 'secondaryEmail', 'mobile_contactability', 'email_contactability')

In [ ]:
com_details_df=com_details_df.join(cust_detail.select('golden_id'),'golden_id','inner')

In [ ]:
com_details_df.persist()
com_details_df.count()

In [ ]:
output_index = 'com_details_df_v2'#cpa_all_prod_fy20,cpa_all_prod_fy21
schema = 'customermart'

import sys
try:
    
     com_details_df.write.option("truncate", "true").format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()
    
    
except Exception as e:
    x = e
    print(x)
else:
    x = 'success'
    print(x)

In [ ]:
com_details_df=com_details_df.drop_duplicates()

In [ ]:
com_details_df=com_details_df.selectExpr('golden_id',"primary_mobile_no","primary_number_status as primary_no_status","primary_number_status_enum as primary_no_status_enum","primary_email as primary_email_id",
                      "primary_email_status as primary_email_status","primary_email_status_enum as primary_email_status_enum",
"preffered_communication_node as preferred_communication_mode","prefferedLanguage as prefered_language",
                      "secondaryMobileNumber as secondary_mobile_no","secondaryEmail as secondary_email_id",
                      "mobile_contactability","email_contactability")

In [ ]:
com_details_df=com_details_df.drop_duplicates()

In [ ]:
com_details_df=com_details_df.select('golden_id', 'primary_mobile_no', 'primary_no_status',col('primary_no_status_enum').cast('int'), 'primary_email_id', 'primary_email_status', col('primary_email_status_enum').cast('int'), 'preferred_communication_mode', 'prefered_language', 'secondary_mobile_no', 'secondary_email_id', 'mobile_contactability', 'email_contactability')

In [ ]:
com_details_df.columns

In [ ]:
target_index='get_spa_communication_details'
print(target_index)

In [ ]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = 'get_spa_communication_details'
#########delete index if already exists########################################################################################################################################################################

In [ ]:
target_doc_type

In [ ]:
target_index_doc_type = target_index +'/'+ target_doc_type

In [ ]:
target_index_doc_type

In [ ]:
######### create mapping##################################################################################################################################################################################################################################################################
mapping_output = {target_doc_type:{"properties": {
"golden_id": {
"type": "keyword"
},
"primary_mobile_no": {
"type": "keyword","index":"false"
},
"primary_no_status": {
"type": "keyword","index":"false"
},
"primary_no_status_enum": {
"type": "keyword","index":"false"
},
"primary_email_id": {
"type": "keyword","index":"false"
},
"primary_email_status": {
"type": "keyword","index":"false"
},
"primary_email_status_enum": {
"type": "keyword","index":"false"
},
"preferred_communication_mode": {
"type": "keyword","index":"false"
},
"prefered_language": {
"type": "keyword","index":"false"
},
"secondary_mobile_no": {
"type": "keyword","index":"false"
},
"secondary_email_id": {
"type": "keyword","index":"false"
},
"mobile_contactability": {
"type":"keyword","index":"false"
},
"email_contactability": {
"type": "keyword","index":"false"
}}
}}



URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index+'/_mapping/'+target_doc_type
headers = {'Content-Type': 'application/json'}
r = requests.put(URL, data=json.dumps(mapping_output), headers=headers)
print(r)

In [ ]:
try:
     com_details_df.write.format('org.elasticsearch.spark.sql').option('es.index.auto.create', 'true')\
        .option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type)\
        .option("es.write.operation","upsert").option("es.mapping.id",'golden_id').mode("append").save() 
except Exception as e :
    x = e
else: 
    x = 200 #success
print(x)

In [ ]:
com_details_df.unpersist()

# healthDetails

In [ ]:
demog_hlt=demog_df.select('golden_id','weight_kg','tobacco_use_indicator','alcohol_use_indicator','height_cm')

In [ ]:
demog_hlt=demog_hlt.withColumn("bmi",col('weight_kg')/((col('height_cm')/100)*(col('height_cm')/100)))

In [ ]:
demog_hlt=demog_hlt.withColumn("bmi",round(col('bmi'),2))

In [ ]:
demog_hlt=header_df.join(demog_hlt,'golden_id','left')

In [ ]:
hlt_p=policy_df.select('source_system_customer_id','reference_num')

hlt_pd=demog_hlt.join(hlt_p,'source_system_customer_id','left')

In [ ]:
hlt_details=hlt_pd.join(pd_df,'reference_num','left')

In [ ]:
hlt_details=hlt_details.repartition('source_system_customer_id')
crgp_df=crgp_df.repartition('source_system_customer_id')

In [ ]:
crgp_df.persist()
crgp_df.count()

In [ ]:
# hlt_details=hlt_details.withColumn('claim_date',coalesce(col('claim_date'),col('claim_reported_date')))

In [ ]:
hlt_details=hlt_details.withColumn('ped',initcap(col('ped')))

In [ ]:
ped=hlt_details.select('golden_id','ped').drop_duplicates()

In [ ]:
ped=ped.groupBy('golden_id').agg(concat_ws(", ",collect_list('ped')).alias('pre_existing_disease'))

In [ ]:
hlt_details=hlt_details.select('golden_id', 'reference_num', 'source_system_customer_id', 'mapping_date', 'weight_kg', 'tobacco_use_indicator', 'alcohol_use_indicator', 'height_cm', 'bmi').drop_duplicates()

In [ ]:
from pyspark.sql.window import Window

hlt_details = hlt_details.withColumn("rn", F.row_number()
        .over(Window.partitionBy('golden_id')
        .orderBy(F.col("mapping_date").desc())))

hlt_details = hlt_details.filter(F.col("rn") == 1).drop("rn")


In [ ]:
hlt_details=hlt_details.join(ped,'golden_id','left')

In [ ]:
hlt_details=hlt_details.join(cust_detail.select('golden_id'),'golden_id','inner')

In [ ]:
hlt_details=hlt_details.join(crgp_df,'source_system_customer_id','left')

In [ ]:
hlt_details.persist()
hlt_details.count()

In [ ]:
hlt_details=hlt_details.selectExpr('golden_id', 'height_cm', 'weight_kg', 'bmi', 'pre_existing_disease as pre_existing_condition', 'tobacco_use_indicator as tobacco_consumer', 'alcohol_use_indicator as alcohol_consumer', 'insured_name as insurred_name', 'loss_description as reason', 'claim_date')

In [ ]:
df_distinct=hlt_details

In [ ]:
df_distinct=df_distinct.withColumn('claim_date',date_format(to_utc_timestamp(col('claim_date'),"UTC"),'yyyy-MM-dd'))

In [ ]:
# df_distinct=df_distinct.withColumn('claim_date',date_format(to_utc_timestamp(col('claim_date'),"UTC"),'yyyy-MM-dd'))

In [ ]:
df_distinct=df_distinct.withColumn('primary_key', concat_ws('',trim(col('golden_id')),trim(col('height_cm')), trim(col('weight_kg')),
                                                       trim(col('bmi')),trim(col('pre_existing_condition')), trim(col('tobacco_consumer')),
                                                       trim(col('alcohol_consumer'))))

In [ ]:
# from pyspark.sql.window import Window

# df_distinct = df_distinct.withColumn("rn", F.row_number()
#         .over(Window.partitionBy("primary_key")
#         .orderBy(F.col("claim_date").desc())))

# df_distinct = df_distinct.filter(F.col("rn") == 1).drop("rn").drop("primary_key")

In [ ]:
df=df_distinct.select('golden_id','insurred_name','reason','claim_date').drop_duplicates()

In [ ]:
df_transformed=df.rdd.map(lambda row:(row.golden_id,{"insurred_name":row.insurred_name,"reason":row.reason,"claim_date":row.claim_date}))
grouped_data=df_transformed.groupByKey().mapValues(list)
df_final=grouped_data.toDF(["golden_id","claim_details"])

In [ ]:
df_main=df_distinct.select(['golden_id', 'height_cm', 'weight_kg', 'bmi', 'pre_existing_condition', 'tobacco_consumer', 'alcohol_consumer']).drop_duplicates()

In [ ]:
df_main=df_main.select(['golden_id', 'height_cm', 'weight_kg', 'bmi', 'pre_existing_condition', 'tobacco_consumer', 'alcohol_consumer']).join(df_final,'golden_id','left')

In [ ]:
df_main.printSchema()

In [ ]:
df_main.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=df_main.count()
records

In [ ]:
records

In [ ]:
crgp_df.unpersist()
hlt_details.unpersist()

In [ ]:
target_index='get_spa_cust_health_details'
print(target_index)

In [ ]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = 'get_spa_cust_health_details'
#########delete index if already exists########################################################################################################################################################################

In [ ]:
target_index_doc_type = target_index +'/'+ target_doc_type
target_index_doc_type

In [ ]:
df_main.columns

In [ ]:
######### create mapping##################################################################################################################################################################################################################################################################
mapping_output = {target_doc_type:{"properties": {
"golden_id": {
"type": "keyword"
},
"height_cm": {
"type": "keyword"
},
"weight_kg": {
"type": "keyword"
},
"bmi": {
"type":"keyword"
},
"pre_existing_condition": {
"type": "keyword"
},
"tobacco_consumer": {
"type": "keyword"
},
"alcohol_consumer": {
"type": "keyword"
},
"claim_details": {
"type": "nested",
    "properties":{
        "insurred_name":{
            "type":"text"
        },
          "reason":{
            "type":"text"
        },
         "claim_date":{
            "type":"keyword"
        }
    }
}
}
}}



URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index+'/_mapping/'+target_doc_type
headers = {'Content-Type': 'application/json'}
r = requests.put(URL, data=json.dumps(mapping_output), headers=headers)
print(r)

In [ ]:
try:
     df_main.write.format('org.elasticsearch.spark.sql').option('es.index.auto.create', 'true')\
        .option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type)\
        .option("es.write.operation","upsert").option("es.mapping.id",'golden_id').mode("append").save() 
except Exception as e :
    x = e
else: 
    x = 200 #success
print(x)

In [ ]:
df_main.unpersist()

# familyDetails

In [ ]:
hcbd=header_df.join(credit_df,'golden_id','left')

In [ ]:
hcbd.columns

In [ ]:
fam_p=policy_df.select('source_system_customer_id','reference_num','assets')

hcbdp=hcbd.join(fam_p,'source_system_customer_id','left')

In [ ]:
hcbdp.columns

In [ ]:
#removing whitespace and special char
pdrdh_df1=pdrdh_df.withColumn('name',regexp_replace(col('name'),"[^a-zA-Z0-9 ]+",""))

In [ ]:
#removing whitespace and special char
pdrdh_df1=pdrdh_df1.withColumn('name_1',lower(regexp_replace(col('name'),"[^a-zA-Z0-9]+","")))

In [ ]:
from pyspark.sql.window import Window

pdrdh_df1 = pdrdh_df1.withColumn("rn", F.dense_rank()
        .over(Window.partitionBy("reference_num")
        .orderBy(F.col("reference_num").desc())))

pdrdh_df1 = pdrdh_df1.filter(F.col("rn") == 1).drop("rn")

In [ ]:
pdrdh_df1=pdrdh_df1.groupBy("reference_num").agg(countDistinct('name_1').alias('family_size'))#need to confirm

In [ ]:
pdrdh_df=pdrdh_df.join(pdrdh_df1,'reference_num','inner')

In [ ]:
pdrdh_df.columns

In [ ]:
hcbdp_risk=hcbdp.join(pdrdh_df,'reference_num','left')

In [ ]:
hcbdp_risk=hcbdp_risk.join(cust_detail.select('golden_id'),'golden_id','inner')

In [ ]:
hcbdp_risk.persist()
hcbdp_risk.count()

In [ ]:
#removing whitespace and special char
# hcbdp_risk=hcbdp_risk.withColumn('name',regexp_replace(col('name'),"[^a-zA-Z0-9 ]+",""))

In [ ]:
#removing whitespace and special char
# hcbdp_risk=hcbdp_risk.withColumn('name_1',lower(regexp_replace(col('name'),"[^a-zA-Z0-9]+","")))

In [ ]:
#fam_size=hcbdp_risk.groupBy("golden_id").agg(countDistinct('name_1').alias('family_size'))#need to confirm

In [ ]:
# fam_size.select('golden_id','family_size').filter(col('golden_id').isin())

In [ ]:
fam_assets=hcbdp.select('golden_id','assets').drop_duplicates()
fam_assets=fam_assets.groupBy('golden_id').agg(concat_ws(", ",collect_list('assets')).alias('assets'))

In [ ]:
hcbdp_risk.columns

In [ ]:
# hcbdp_risk=hcbdp_risk.join(fam_size,'golden_id','left')
hcbdp_risk=hcbdp_risk.join(fam_assets,'golden_id','left').drop(hcbdp_risk.assets)

In [ ]:
hcbdp_risk.columns

In [ ]:
hcbdp_risk=hcbdp_risk.drop_duplicates()

In [ ]:
hcbdp_risk_f=hcbdp_risk.selectExpr('golden_id', 'reference_num', 'source_system_customer_id', 'source as credit_score_source', 'scoredate as credit_score_update_date', 'score as credit_score', 'income as gross_annual_income','income as gross_annual_incomeBucket', 'assets as assets', 'relation', 'name', 'dob', 'family_size as familySize',"'-' as anniversarydate")

In [ ]:
hcbdp_risk_f = hcbdp_risk_f.withColumn('gross_annual_incomeBucket', when((col('gross_annual_income') <=500000), lit('Upto 5 lacs'))\
                             .when((col('gross_annual_income') >500000) & (col('gross_annual_income') <=700000), lit('5 - 7 lacs'))\
                             .when((col('gross_annual_income') >700000) & (col('gross_annual_income') <=1000000), lit( '7 - 10 lacs'))\
                             .when((col('gross_annual_income') >1000000) & (col('gross_annual_income') <=1500000), lit('10 -15 lacs'))\
                             .when((col('gross_annual_income') >1500000) & (col('gross_annual_income') <=2000000), lit('15 - 20 lacs'))\
                             .when((col('gross_annual_income') >2000000) & (col('gross_annual_income') <=3000000), lit('20 - 30 lacs'))\
                             .when((col('gross_annual_income') >3000000) & (col('gross_annual_income') <=4000000), lit('30 - 40 lacs'))\
                             .when((col('gross_annual_income') >4000000) & (col('gross_annual_income') <=6000000), lit('40 - 60 lacs'))\
                             .when((col('gross_annual_income') >6000000) & (col('gross_annual_income') <=8000000), lit('60 - 80 lacs'))\
                             .when((col('gross_annual_income') >8000000) & (col('gross_annual_income') <=10000000), lit('80 lacs to 1 cr'))\
                             .when((col('gross_annual_income') >=10000000), lit('Above 1 cr'))\
                                .otherwise(col('gross_annual_income')))

In [ ]:
df_distinct=hcbdp_risk_f.withColumn('dob',to_date(col('dob'),'dd/MM/yyyy'))

In [ ]:
df_distinct=df_distinct.withColumn('dob',date_format(to_utc_timestamp(col('dob'),"UTC"),'yyyy-MM-dd'))

In [ ]:
df_distinct=df_distinct.withColumn('primary_key', concat_ws('',trim(col('golden_id')),trim(col('familySize')), trim(col('credit_score')),
                                                       trim(col('credit_score_update_date')),trim(col('credit_score_source')), trim(col('assets')),
                                                       trim(col('gross_annual_income')),trim(col('gross_annual_incomeBucket')), trim(col('anniversarydate'))))

In [ ]:
df_distinct.columns

In [ ]:
df_temp=df_distinct.select('golden_id','relation', 'name', 'dob').drop_duplicates()

In [ ]:
df_transformed=df_temp.rdd.map(lambda row:(row.golden_id,{"name":row.name,"relationship":row.relation,"dob":row.dob}))

In [ ]:
grouped_data=df_transformed.groupByKey().mapValues(list)
df_final=grouped_data.toDF(["golden_id","member_details"])

In [ ]:
df=df_distinct.select("golden_id","familySize","gross_annual_income","credit_score","credit_score_source","credit_score_update_date",
                         "assets","anniversaryDate","gross_annual_incomeBucket").drop_duplicates()

In [ ]:
df_main=df.join(df_final,'golden_id','left')

In [ ]:
df_main=df_main.withColumn('assets',regexp_replace(col('assets'),r', -','')).withColumn('assets',regexp_replace(col('assets'),r'-, ',''))\
.withColumn('assets',trim(col('assets')))

In [ ]:
from pyspark.sql.window import Window

df_main = df_main.withColumn("rn", F.row_number()
        .over(Window.partitionBy("golden_id")
        .orderBy(F.col("golden_id").desc())))

df_main = df_main.filter(F.col("rn") == 1).drop("rn")

In [ ]:
df_main.persist()
df_main.count()

In [ ]:
hcbdp_risk.unpersist()

In [ ]:
target_index='get_spa_member_score_details'
print(target_index)

In [ ]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = 'get_spa_member_score_details'
#########delete index if already exists########################################################################################################################################################################

In [ ]:
target_doc_type

In [ ]:
target_index_doc_type = target_index +'/'+ target_doc_type
target_index_doc_type

In [ ]:
######### create mapping##################################################################################################################################################################################################################################################################
mapping_output = {target_doc_type:{"properties": {
"golden_id": {
"type": "keyword"
},
"familySize": {
"type": "keyword","index":"false"
},
"credit_score": {
"type": "keyword","index":"false"
},
"credit_score_update_date": {
"type":"keyword","index":"false"
},
"credit_score_source": {
"type": "keyword","index":"false"
},
"gross_annual_income": {
"type": "keyword","index":"false"
},
"assets": {
"type": "keyword","index":"false"
},
"gross_annual_incomeBucket": {
"type": "keyword","index":"false"
},
"anniversarydate": {
"type":"keyword","index":"false"
},
"member_details": {
"type": "nested",
    "properties":{
        "name":{
            "type":"text","index":"false"
        },
          "relationship":{
            "type":"text","index":"false"
        },
         "dob":{
            "type":"keyword","index":"false"
        }
    }
}
}
}}



URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index+'/_mapping/'+target_doc_type
headers = {'Content-Type': 'application/json'}
r = requests.put(URL, data=json.dumps(mapping_output), headers=headers)
print(r)

In [ ]:
try:
     df_main.write.format('org.elasticsearch.spark.sql').option('es.index.auto.create', 'true')\
        .option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type)\
        .option("es.write.operation","upsert").option("es.mapping.id","golden_id").mode("append").save() 
except Exception as e :
    x = e
else: 
    x = 200 # success
print(x)

In [ ]:
hcbdp_risk_f.unpersist()
df_main.unpersist()

In [ ]:
import io
import pandas as pd
import xlsxwriter
import sys
import smtplib
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
import pandas as pd
from sqlalchemy import create_engine
from smtplib import SMTP

if x!=200:
    status='FAILED'
else:
    status="SUCCESS"

subject=f"Automated Mail-SPA PART-2 Job Status : {status}"

if x==200:
    recipients = ['nishant5.addenki@tataaig.com','devnandan.TataIndustries@tataaig.com','mayur.TataIndustries@tataaig.com','Ajeet3.Jaiswal@tataaig.com'] 
    emaillist = [elem.strip().split(',') for elem in recipients]
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = 'customercoe@tataaig.com'


    html = """\
    <html>
      <head></head>
      <body>
        SPA PART-2 job status is: Successful
      </body>
        <p>This is automated mail, please don't reply. Thanks</p>
    </html>
    """

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    server = smtplib.SMTP("tataaig-com.mail.protection.outlook.com",25)
    server.sendmail(msg['From'], emaillist , msg.as_string())
else:
      
    recipients = ['nishant5.addenki@tataaig.com','devnandan.TataIndustries@tataaig.com','mayur.TataIndustries@tataaig.com','Ajeet3.Jaiswal@tataaig.com'] 
    emaillist = [elem.strip().split(',') for elem in recipients]
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = 'customercoe@tataaig.com'


    html = """\
    <html>
      <head></head>
      <body>
        CPA job status is: Failed
      </body>
        <p>This is automated mail, please don't reply. Thanks</p>
    </html>
    """

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    server = smtplib.SMTP("tataaig-com.mail.protection.outlook.com",25)
    server.sendmail(msg['From'], emaillist , msg.as_string())

In [ ]:
spark.stop()